In [1]:
from __future__ import print_function

In [2]:
import modeled
from modeled.netconf import YANGContainer, YANGModule, rpc

In [3]:
class Input(modeled.object):
    state = modeled.member[int]()
    symbol = modeled.member[str]()

In [4]:
class Output(modeled.object):
    state = modeled.member[int]()
    symbol = modeled.member[str]()
    head_move = modeled.member[str]['L', 'R']('R')

In [5]:
class Transition(modeled.object):
    input = modeled.member[Input]()
    output = modeled.member[Output]()

    def __init__(self, input, output):
        """Expects `input` as `output` as mappings.
        """
        self.input = Input(
            # modeled.object.__init__ supports **kwargs
            # for initializing modeled.member values
            **dict(input))
        self.output = Output(**dict(output))

In [6]:
class TuringMachine(modeled.object):
    state = modeled.member[int]()
    head_position = modeled.member[int]()
    tape = modeled.member.list[str]()
    transitions = modeled.member.dict[str, Transition]()

    def __init__(self, transitions):
        modeled.object.__init__(self, transitions={})
        transitions = dict(transitions)
        for name, (input, output) in transitions.items():
            self.transitions[name] = Transition(input, output)

    def run(self):
        while True:
            if 0 <= self.head_position < len(self.tape):
                symbol = self.tape[self.head_position]
            else:
                symbol = None
            for name, trans in self.transitions.items():
                if self.state == trans.input.state and symbol == trans.input.symbol:
                    print(self.tape, end=" ")
                    if trans.output.state is not None:
                        self.state = trans.output.state
                    if trans.output.symbol is not None:
                        self.tape[self.head_position] = trans.output.symbol
                    self.head_position += {'L': -1, 'R': 1}[trans.output.head_move]
                    print("-->", self.tape)
                    break
            else:
                break

In [7]:
class TM(YANGModule[TuringMachine]):

    @rpc(argtypes={'tape_content': str})
    def initialize(self, tape_content):
        """Initialize the Turing Machine.
        """
        self.state = 0
        self.head_position = 0
        self.tape = tape_content

    @rpc(argtypes={})
    def run(self):
        """Start the Turing Machine operation.
        """
        TuringMachine.run(self)

In [8]:
%%file transitions.yaml

"left summand":
  - {state:    0, symbol:    1}
  - {state: null, symbol: null}
"separator":
  - {state:    0, symbol:    0}
  - {state:    1, symbol:    1}
"right summand":
  - {state:    1, symbol:    1}
  - {state: null, symbol: null}
"right end":
  - {state:    1, symbol: null}
  - {state:    2, symbol: null, head_move: L}
"write separator":
  - {state:    2, symbol:    1}
  - {state:    3, symbol:    0, head_move: L}
"go home":
  - {state:    3, symbol:    1}
  - {state: null, symbol: null, head_move: L}
"final step":
  - {state:    3, symbol: null}
  - {state:    4, symbol: null}

Overwriting transitions.yaml


In [9]:
import yaml
with open('transitions.yaml') as f:
    PROGRAM = yaml.load(f)

In [10]:
tm = TM(PROGRAM)

In [11]:
tm.initialize('101')

In [12]:
tm.run()

['1', '0', '1'] --> ['1', '0', '1']
['1', '0', '1'] --> ['1', '1', '1']
['1', '1', '1'] --> ['1', '1', '1']
['1', '1', '1'] --> ['1', '1', '1']
['1', '1', '1'] --> ['1', '1', '0']
['1', '1', '0'] --> ['1', '1', '0']
['1', '1', '0'] --> ['1', '1', '0']
['1', '1', '0'] --> ['1', '1', '0']


In [13]:
TM_YANG = TM.to_yang(namespace='https://netconf.modeled.io/turing-machine')
print(TM_YANG)

module turing-machine {
  namespace "https://netconf.modeled.io/turing-machine";
  prefix tm;

  revision 2015-10-25;

  container turing-machine {
    leaf state {
      type int64;
    }
    leaf head-position {
      type int64;
    }
    list tape {
      key "index";
      leaf index {
        type int64;
      }
      leaf item {
        type string;
      }
    }
    list transitions {
      key "key";
      leaf key {
        type string;
      }
      container item {
        container input {
          leaf state {
            type int64;
          }
          leaf symbol {
            type string;
          }
        }
        container output {
          leaf state {
            type int64;
          }
          leaf symbol {
            type string;
          }
          leaf head-move {
            type string;
          }
        }
      }
    }
  }
  rpc initialize {
    description
      "Initialize the Turing Machine.";
    input {
      leaf tape-content {
        ty

In [14]:
with open('turing-machine.yang', 'w') as f:
    f.write(TM_YANG)

In [15]:
!pyang -f tree turing-machine.yang

module: turing-machine
   +--rw turing-machine
      +--rw state?           int64
      +--rw head-position?   int64
      +--rw tape* [index]
      |  +--rw index    int64
      |  +--rw item?    string
      +--rw transitions* [key]
         +--rw key     string
         +--rw item
            +--rw input
            |  +--rw state?    int64
            |  +--rw symbol?   string
            +--rw output
               +--rw state?       int64
               +--rw symbol?      string
               +--rw head-move?   string
rpcs:
   +---x initialize
   |  +---w input
   |     +---w tape-content?   string
   +---x run


In [16]:
server = tm.serve(port=12345, host_key='key', username='user', password='password')

In [17]:
from netconf.client import NetconfSSHSession

In [18]:
client = NetconfSSHSession('localhost', port=12345, username='user', password='password')

In [ ]:
client.send_rpc('<run/>')

In [20]:
tm.state

4